In [ ]:
from py_eddy_tracker.data import get_remote_sample, get_path
from py_eddy_tracker.observations.network import NetworkObservations
import numba as nb
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import panel as pn
import param
import cartopy.crs as ccrs

hv.extension('bokeh', logo=False)
pn.extension()
proj = ccrs.PlateCarree()



# essai dans une classe

In [ ]:
proj = ccrs.PlateCarree()
class MyNetworkObservations(NetworkObservations):
    def visu(self, contour_type = "speed"):
        if contour_type == "speed":
            contour_lon = self.contour_lon_s
            contour_lat = self.contour_lat_s
            
        elif contour_type == "effective":
            contour_lon = self.contour_lon_e
            contour_lat = self.contour_lat_e
            
        else:
            print("bad contour_type")
            return None
        
#         effective, speed
        full_time = self.time
        timed = np.unique(full_time)

#         slider = pn.widgets.IntSlider(name='date', start=int(timed[0]), end=int(timed[-1]), step=1, value=int(timed[0]))
        slider = pn.widgets.DiscretePlayer(name="date", options=list(timed), value=timed[0])

        def select(tn, lon, lat):
            mask = full_time == tn

            sub_lon = lon[mask]
            sub_lat = lat[mask]

            _lon, _lat = flatten_line_matrix(sub_lon, sub_lat)
            return np.array([_lon, _lat]).T

        extents = (-4, 34, 30, 45)

        @pn.depends(slider)
        def update(*args, **kwargs):
            print("args = ", args)
            print("kwargs = ", kwargs)
            date = kwargs["value"]
            return gv.Path([select(date, contour_lon, contour_lat)], crs=proj).opts(color='white')

        @pn.depends(slider)
        def plot_date(value):
            return pn.pane.HTML(f"<h1>date = {np.datetime64('1950-01-01')+np.timedelta64(value)}</h1>")

#            gf.land.options(scale='50m')
#          * gf.coastline.options(scale='50m')
        return pn.Column(
            pn.Row(slider, plot_date),
            (
                 gv.tile_sources.ESRI             
                 * hv.DynamicMap(update)
                * gv.Points([[extents[0], extents[2]], [extents[1], extents[3]]])
            ).opts(width=900, height=600)
        )

with Timer("chargement fichier"):
    fichier = get_path("network_med.nc")
    network = MyNetworkObservations.load_file(fichier, include_vars=["time", "track", "speed_contour_longitude", "speed_contour_latitude"])

network.remove_trash().visu("speed")

In [ ]:
gv.help(gv.Path())